In [1]:
import torch
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from MelodyLSTMEmbV2 import MelodyLSTMEmb

# Getting data

In [2]:
class ChordMelodyDataset(torch.utils.data.Dataset):
    def __init__(self, track_count, compass_count_path, X_path, y_path, window_size=10):
        self.window_size = window_size
        
        print('Loading compass count')
        self.compass_count = pd.read_csv(compass_count_path, names=['tid', 'compass_count'], nrows=track_count)
        self.cc = self.compass_count['compass_count'].to_numpy()
        self.cc_w = self.compass_count['compass_count'].to_numpy() - window_size
        self.cum_cc = np.cumsum(self.cc)
        self.cum_cc_w = np.cumsum(self.cc_w)
        
        self.cum_cc = np.append([0], self.cum_cc)
        self.cum_cc_w = np.append([0], self.cum_cc_w)

        
        print('Loading X')
        self.X = pd.read_csv(X_path, names=['n'+str(i) for i in range(36)] + ['chord', 'prev'], nrows=self.cum_cc[-1])
        
        print('Loading y')
        self.y = pd.read_csv(y_path, names=['n'+str(i) for i in range(36)], nrows=self.cum_cc[-1])
        
    def __len__(self):
        return self.cum_cc_w[-1]
    
    def __getitem__(self, idx):
        bucket = np.searchsorted(self.cum_cc_w, idx, side='right')
        # print(bucket)
        delta = idx - self.cum_cc_w[bucket - 1]
        # print(delta)
        idx_start = self.cum_cc[bucket - 1]
        # print(idx_start)
        idx = idx_start + delta
        
        # print(idx)
        x_data = torch.tensor(self.X.iloc[idx : idx + self.window_size].to_numpy(dtype=float))
        y_data = torch.tensor(self.y.iloc[idx : idx + self.window_size].to_numpy(dtype=float))

        return x_data.float(), y_data.float()

# Training

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [5]:
dataset = ChordMelodyDataset(10000, '../data/compass_count2.csv', '../data/X2.csv', '../data/y2.csv', window_size=32)

Loading compass count
Loading X
Loading y


In [6]:
len(dataset)

7626896

In [7]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=256)

In [8]:
num_epochs = 100
learning_rate = 1e-6

input_size = 56 #number of features
hidden_size = 512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

output_size = 36 #number of output classes 

In [9]:
mlstm = MelodyLSTMEmb(input_size, hidden_size, output_size, 
                   num_layers, device, threshold=0.6)
mlstm = mlstm.to(device)

In [10]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(mlstm.parameters(), lr=learning_rate) 

In [11]:
for epoch in range(num_epochs):
    for X, y in tqdm(data_loader):
        X = X.cuda()
        y = y.cuda()
        outputs = mlstm.forward(X) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
        
        # obtain the loss function
        # print(outputs.shape, y[:,-1,:].shape)
        loss = criterion(outputs, y[:,-1,:])

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
        
    torch.save(mlstm.state_dict(), f'../models/muse2_t10000_bce_w32/checkpoint-{epoch:02}.pth')    
    print("Epoch: %d, loss: %1.8f" % (epoch, loss.item())) 

100%|████████████████████████████████████████████████████████████████████████████| 29793/29793 [29:51<00:00, 16.63it/s]


Epoch: 0, loss: 0.28456211


100%|████████████████████████████████████████████████████████████████████████████| 29793/29793 [29:57<00:00, 16.57it/s]


Epoch: 1, loss: 0.28321105


100%|████████████████████████████████████████████████████████████████████████████| 29793/29793 [29:14<00:00, 16.98it/s]


...

Epoch: 45, loss: 0.16628347


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29793/29793 [29:28<00:00, 16.85it/s]


Epoch: 46, loss: 0.16535012


 34%|██████████████████████████████████████████████████▎                                                                                                  | 10059/29793 [09:56<19:29, 16.87it/s]


KeyboardInterrupt: 